# Directed Graphs, Degree Distribution and Sampling
### Contents:
1. Creating simple graphs
2. Importing a dataset with directed and weighted edges 
3. Connectivity in Directed Graphs
4. Degree Distribution
5. Converting the graph to an undirected 
6. Sampling Revisited

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

### Creating a simple undirected graph

In [ ]:
edges = [(1, 2), (1, 6), (2, 3), (2, 4), (2, 6), 
         (3, 4), (3, 5), (4, 8), (4, 9), (6, 7)]

In [ ]:
G_basic = nx.Graph()
G_basic.add_edges_from(edges)
nx.draw_networkx(G_basic, with_label = True)

### Creating a simple undirected weighted graph

In [ ]:
G_weighted = nx.Graph()
  
edges = [(1, 2, 19), (1, 6, 15), (2, 3, 6), (2, 4, 10), 
         (2, 6, 22), (3, 4, 51), (3, 5, 14), (4, 8, 20),
         (4, 9, 42), (6, 7, 30)]
  
G_weighted.add_weighted_edges_from(edges)
nx.draw_networkx(G_weighted, with_labels = True)

In [ ]:
print(list(G_weighted.edges(data = True)))

### Creating a simple directed graph

In [ ]:
G_directed = nx.DiGraph()
G_directed.add_edges_from([(1, 1), (1, 7), (2, 1), (2, 2), (2, 3), 
                  (2, 6), (3, 5), (4, 3), (5, 4), (5, 8),
                  (5, 9), (6, 4), (7, 2), (7, 6), (8, 7)])
  
plt.figure(figsize =(9, 9))
nx.draw_networkx(G_directed, with_label = True, node_color ='green')

### Importing a dataset with directed and weighted edges 
#### About the dataset - Example of a trust distrust network
- Who-trusts-whom network of people who trade using Bitcoin on the platform: Bitcoin Alpha. 
- Bitcoin users are anonymous, there is a need to maintain a record of users' reputation to prevent transactions with fraudulent and risky users. 
- Members of Bitcoin Alpha rate other members in a scale of -10 (total distrust) to +10 (total trust) in steps of 1. <br> 

For more datasets visit https://snap.stanford.edu/data/index.html

In [ ]:
fname = 'soc-sign-bitcoinalpha.csv'
cols = ['source', 'target', 'weight', 'time']
df = pd.read_csv(fname, names=cols, header=None)
df['time'] = pd.to_datetime(df.time * 1e9)
df.shape

In [ ]:
df.head(10)

In [ ]:
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr = 'weight' , create_using=nx.DiGraph())
n,e = G.order(), G.size()
ave_degree = float(e)/n
trans=nx.transitivity(G)

In [ ]:
print ("Nodes: ", n)
print ("Edges: ", e)
print ("Average degree: ", ave_degree)
print ("Trasitivity: ", trans)

In [ ]:
G.number_of_nodes() # also g.order()

In [ ]:
G.number_of_edges() # also g.size()

### Before we move further, we will review the dictionary concept in Python.

In [ ]:
# Python dictionaries:

# Keys and values can be of any data type
fruit_dict = {"apple":1, "orange":[0.23,0.11], 42:True}

# Can retrieve the keys and values as Python lists (vector)
fruit_dict.keys() 

In [ ]:
# Or create a (key,value) tuple
fruit_dict.items() 

Any NetworkX graph behaves like a Python dictionary with nodes as primary keys.

### Connectivity in Directed Graphs

In [ ]:
nx.is_strongly_connected(G)

In [ ]:
nx.number_strongly_connected_components(G)

In [ ]:
nx.is_weakly_connected(G)

In [ ]:
nx.number_weakly_connected_components(G)

In [ ]:
[len(Gc) for Gc in sorted(nx.strongly_connected_component_subgraphs(G), key=len, reverse=True)]

In [ ]:
Gc = max(nx.strongly_connected_component_subgraphs(G), key=len)

In [ ]:
Gc.number_of_nodes(),Gc.number_of_edges()

In [ ]:
[len(c) for c in sorted(nx.weakly_connected_component_subgraphs(G),key=len, reverse=True)]

In [ ]:
Gcw = max(nx.weakly_connected_component_subgraphs(G), key=len)
Gcw.number_of_nodes(),Gcw.number_of_edges()

### Degree Distribution

<img src="log.png" alt="log scale" title="Log Explained" />

In [ ]:
def degree_histogram_directed(G, in_degree=False, out_degree=False):
    
    """Return a list of the frequency of each degree value.

    Parameters
    ----------
    G : Networkx graph
    in_degree : bool
    out_degree : bool

    Returns
    -------
    hist : list 
        A list of frequencies of degrees.
        The degree values are the index in the list.

    """
    nodes = G.nodes()
    if in_degree:
        in_degree = dict(G.in_degree())
        degseq=[in_degree.get(k,0) for k in nodes]
        
    elif out_degree:
        out_degree = dict(G.out_degree())
        degseq=[out_degree.get(k,0) for k in nodes]
        
    else:
        degseq=[v for k, v in G.degree()]
        
    dmax=max(degseq)+1
    freq= [ 0 for d in range(dmax) ]
    for d in degseq:
        freq[d] += 1
    return freq

In [ ]:
in_degree_freq = degree_histogram_directed(G, in_degree=True)
out_degree_freq = degree_histogram_directed(G, out_degree=True)
degrees = range(len(in_degree_freq))
plt.figure(figsize=(12, 8)) 
plt.loglog(range(len(in_degree_freq)), in_degree_freq, 'go-', label='in-degree') 
plt.loglog(range(len(out_degree_freq)), out_degree_freq, 'bo-', label='out-degree')
plt.xlabel('Degree')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(12, 8)) 
plt.loglog(range(len(in_degree_freq)), in_degree_freq, 'gD-.', label='in-degree') 
plt.loglog(range(len(out_degree_freq)), out_degree_freq, 'ro--', label='out-degree')
plt.xlabel('Degree')
plt.ylabel('Frequency')

See more plotting options in this link.
https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

### Converting the graph to an undirected 

In [ ]:
# Now, we will convert the graph to an undirected network and extract the giant connected component
# then we will compute node centrality measures.

UG = G.to_undirected()

UG_componenets = (UG.subgraph(c) for c in nx.connected_components(UG))
UG_gc = list(UG_componenets)[0]

In [ ]:
num_nodes = UG_gc.order()
num_nodes

### Sampling Revisited

In [ ]:
# let's see how hte nodes are represented in Bitcoin dataset
sorted(UG_gc)[:10]

Note that they are consecutive integers however they don't start with zero. So we have to relabel them.

In [ ]:
mapping = dict(zip(UG_gc, range(0, num_nodes)))
UG_gc = nx.relabel_nodes(UG_gc, mapping)
sorted(UG_gc)[:10]

In [ ]:
from littleballoffur import ForestFireSampler
sampler = ForestFireSampler(number_of_nodes=500)

new_sample = sampler.sample(UG_gc)

In [ ]:
n,e = new_sample.order(), new_sample.size()
ave_degree = float(e)/n
trans = nx.transitivity(new_sample)
print ("Nodes: ", n)
print ("Edges: ", e)
print ("Average degree: ", ave_degree)
print ("Transitivity: ", trans)

In [ ]:
degree_freq = degree_histogram_directed(new_sample)
degrees = range(len(degree_freq))
plt.figure(figsize=(12, 8)) 
plt.loglog(degrees, degree_freq, 'go-', label='degree') 
plt.xlabel('Degree')
plt.ylabel('Frequency')

# Your Turn

In [ ]:
# 1. Using G_directed graph created above, find the nuber of nodes and edges in this graph. to answer 


In [ ]:
# 2. Check whether the graph is strongly/weakly connected. (2 statements)


In [ ]:
# 3. Find in-degree, out-degree of all nodes.


In [ ]:
# 4. Find the number of strongly connected components in G_directed.


In [ ]:
# 5. Find the number of weakly connected components in G_directed.


6. Explain in your own words what is a degree distribution?

7. Explain in your own words what is a log scale and why do we use it?

8. Can you give another real-life example of trust distrust network?

9. Why did we find the degree distribution and the transititvity of the sampled graph?


10. Given the bitcoin dataset above think of one interesting information you can obtain from this dataset.

**Bonus question:**
Can you come up with an algorithm that uses the Littlebaloffur library to sample a directed graph? Just write the psudo code (plain English steps).